## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,30.36,80,99,7.31,overcast clouds
1,1,Mataura,NZ,-46.1927,168.8643,65.48,39,4,1.68,clear sky
2,2,Busselton,AU,-33.6500,115.3333,65.05,53,0,18.63,clear sky
3,3,Kapaa,US,22.0752,-159.3190,71.92,82,90,16.11,heavy intensity rain
4,4,Taurage,LT,55.2522,22.2897,9.05,94,16,4.18,few clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
13,13,Isangel,VU,-19.5500,169.2667,83.35,72,5,1.54,clear sky
14,14,Rikitea,PF,-23.1203,-134.9692,76.53,71,68,8.32,broken clouds
16,16,Cabo San Lucas,MX,22.8909,-109.9124,78.51,69,1,3.44,clear sky
19,19,Mahebourg,MU,-20.4081,57.7000,78.26,72,11,10.47,few clouds
21,21,Cotonou,BJ,6.3654,2.4183,81.82,82,94,11.79,overcast clouds
28,28,Vaini,TO,-21.2000,-175.2000,82.56,83,20,11.50,few clouds
33,33,Avarua,CK,-21.2078,-159.7750,80.65,89,75,3.44,light rain
39,39,Ixtapa,MX,20.7000,-105.2000,79.83,78,40,5.75,scattered clouds
43,43,Vao,NC,-22.6667,167.4833,78.39,76,0,5.21,clear sky
45,45,Alice Springs,AU,-23.7000,133.8833,76.55,65,40,10.36,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                172
City                   172
Country                172
Lat                    172
Lng                    172
Max Temp               172
Humidity               172
Cloudiness             172
Wind Speed             172
Current Description    172
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# N/A

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Isangel,VU,83.35,clear sky,-19.5500,169.2667,
14,Rikitea,PF,76.53,broken clouds,-23.1203,-134.9692,
16,Cabo San Lucas,MX,78.51,clear sky,22.8909,-109.9124,
19,Mahebourg,MU,78.26,few clouds,-20.4081,57.7000,
21,Cotonou,BJ,81.82,overcast clouds,6.3654,2.4183,
28,Vaini,TO,82.56,few clouds,-21.2000,-175.2000,
33,Avarua,CK,80.65,light rain,-21.2078,-159.7750,
39,Ixtapa,MX,79.83,scattered clouds,20.7000,-105.2000,
43,Vao,NC,78.39,clear sky,-22.6667,167.4833,
45,Alice Springs,AU,76.55,scattered clouds,-23.7000,133.8833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

In [21]:
clean_hotel_df = hotel_df
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
13,Isangel,VU,83.35,clear sky,-19.5500,169.2667,Tanna Lodge
14,Rikitea,PF,76.53,broken clouds,-23.1203,-134.9692,People ThankYou
16,Cabo San Lucas,MX,78.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
19,Mahebourg,MU,78.26,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
21,Cotonou,BJ,81.82,overcast clouds,6.3654,2.4183,Novotel Cotonou Orisha
...,...,...,...,...,...,...,...
669,Pemba,MZ,79.07,scattered clouds,-12.9740,40.5178,Guesthouse Samotina
670,Robertsport,LR,81.03,light rain,6.7533,-11.3686,Kwepunha Retreat
676,Garissa,KE,76.64,scattered clouds,-0.4536,39.6401,Sharif Plaza Hotel
681,Ambunti,PG,87.93,broken clouds,-4.2365,142.8448,Ambunti Lodge


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))